<a href="https://colab.research.google.com/github/Durso16/head_bet_/blob/main/relatorio_completo_bet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ipywidgets pandas matplotlib numpy
!jupyter nbextension enable --py widgetsnbextension

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [13]:
import numpy as np
import pandas as pd
from google.colab import files

# Passo 2: Carregar o arquivo de odds
file_path = '/content/odds_rodada24.csv'  # Substitua pelo caminho correto do arquivo de odds
odds_df = pd.read_csv(file_path)

# Função para calcular as apostas ideais e verificar lucro
def calcular_apostas_e_lucro(odds_a, odds_e, odds_b, max_investimento=100):
    investimento_a = max_investimento / (1 + (odds_b / odds_a) + (odds_e / odds_a))
    investimento_b = (odds_b / odds_a) * investimento_a
    investimento_e = (odds_e / odds_a) * investimento_a

    retorno_a = investimento_a * odds_a
    retorno_b = investimento_b * odds_b
    retorno_e = investimento_e * odds_e

    lucro_a = retorno_a - max_investimento
    lucro_b = retorno_b - max_investimento
    lucro_e = retorno_e - max_investimento

    return investimento_a, investimento_e, investimento_b, lucro_a, lucro_e, lucro_b

# Função para verificar as condições e gerar resultados
def verificar_condicoes(odds_df, max_prejuizo_empate=20):
    resultados = []

    for i, row in odds_df.iterrows():
        time_a = row['TimeA']
        odds_a = row['Odds_TimeA']
        odds_e = row['Odds_Empate']
        time_b = row['TimeB']
        odds_b = row['Odds_TimeB']

        investimento_a, investimento_e, investimento_b, lucro_a, lucro_e, lucro_b = calcular_apostas_e_lucro(odds_a, odds_e, odds_b)

        resultado_jogo = {
            "Jogo": f"{time_a} vs {time_b}",
            "investimento_a": investimento_a,
            "investimento_b": investimento_b,
            "investimento_e": investimento_e,
            "lucro_a": lucro_a,
            "lucro_b": lucro_b,
            "lucro_e": lucro_e,
            "prejuizo_empate": -lucro_e
        }

        resultados.append(resultado_jogo)

    # Ordenar os resultados pelo prejuízo no empate (mais próximo da tolerância primeiro)
    resultados.sort(key=lambda x: abs(x['prejuizo_empate'] - max_prejuizo_empate))

    return resultados

# Função para exibir e salvar resultados no formato desejado
def exibir_e_salvar_resultados(resultados):
    with open("resultados.txt", "w") as f:
        jogos_satisfatorios = []
        jogos_com_prejuizo_admissivel = []

        for resultado in resultados:
            formato = (
                f"Cenário             Aposta no Time (R$)      Retorno Líquido (R$)     Lucro se o Time (R$)\n"
                f"--------------------------------------------------------------------------------\n"
                f"Vitória {resultado['Jogo'].split(' vs ')[0]:<15}R${resultado['investimento_a']:<20.2f}R${resultado['investimento_a'] + resultado['lucro_a']:<23.2f}R${resultado['lucro_a']:<.2f}\n"
                f"Vitória {resultado['Jogo'].split(' vs ')[1]:<15}R${resultado['investimento_b']:<20.2f}R${resultado['investimento_b'] + resultado['lucro_b']:<23.2f}R${resultado['lucro_b']:<.2f}\n"
                f"Empate              R${resultado['investimento_e']:<20.2f}R${resultado['investimento_e'] + resultado['lucro_e']:<23.2f}R${resultado['lucro_e']:<.2f}\n"
            )
            print(formato)
            f.write(formato + "\n")

            if resultado['lucro_a'] > 0 and resultado['lucro_b'] > 0 and resultado['lucro_e'] > 0:
                jogos_satisfatorios.append(resultado['Jogo'])
            elif resultado['lucro_a'] > 0 and resultado['lucro_b'] > 0 and resultado['prejuizo_empate'] <= 20:
                jogos_com_prejuizo_admissivel.append(resultado['Jogo'])

        if jogos_satisfatorios:
            mensagem = "Jogos que satisfazem as condições de lucro positivo em todos os cenários:\n"
            print(mensagem)
            f.write(mensagem)
            for jogo in jogos_satisfatorios:
                print(jogo)
                f.write(f"{jogo}\n")

        if jogos_com_prejuizo_admissivel:
            mensagem = "\nJogos que satisfazem as condições de prejuízo admissível no empate:\n"
            print(mensagem)
            f.write(mensagem)
            for jogo in jogos_com_prejuizo_admissivel:
                print(jogo)
                f.write(f"{jogo}\n")

        if not jogos_satisfatorios and not jogos_com_prejuizo_admissivel:
            mensagem = "Nenhum jogo satisfaz as condições para lucro positivo em todos os cenários ou prejuízo admissível no empate."
            print(mensagem)
            f.write(mensagem + "\n")

    print("\nArquivo 'resultados.txt' gerado com sucesso!")

    # Download automático do arquivo na máquina local
    files.download("resultados.txt")

# Função principal para executar a análise
def executar_analise(tolerancia):
    resultados = verificar_condicoes(odds_df, max_prejuizo_empate=tolerancia)
    exibir_e_salvar_resultados(resultados)

# Defina a tolerância de prejuízo no empate
tolerancia = 20  # Ajuste este valor conforme necessário

# Executar a análise
executar_analise(tolerancia)









Cenário             Aposta no Time (R$)      Retorno Líquido (R$)     Lucro se o Time (R$)
--------------------------------------------------------------------------------
Vitória Atlético MineiroR$17.82               R$-50.10                 R$-67.92
Vitória Fluminense     R$49.50               R$197.03                 R$147.52
Empate              R$32.67               R$40.50                  R$7.82

Cenário             Aposta no Time (R$)      Retorno Líquido (R$)     Lucro se o Time (R$)
--------------------------------------------------------------------------------
Vitória Atlético GoianienseR$23.86               R$-26.02                 R$-49.89
Vitória Juventude      R$39.77               R$78.98                  R$39.20
Empate              R$36.36               R$52.73                  R$16.36

Cenário             Aposta no Time (R$)      Retorno Líquido (R$)     Lucro se o Time (R$)
--------------------------------------------------------------------------------
Vitória Forta

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>